In [1]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from common import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|███████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 8745.45it/s]


In [3]:
dataset[:10]

[('09.05.98', '1998-05-09'),
 ('11/10/19', '2019-11-10'),
 ('thursday september 10 1970', '1970-09-10'),
 ('monday august 19 2024', '2024-08-19'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('07 mar 1983', '1983-03-07'),
 ('22 may 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('wednesday september 8 1999', '1999-09-08')]

In [4]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [5]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 09.05.98
Target date: 1998-05-09

Source after preprocessing (indices): [ 3 12  1  3  8  1 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [6]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [7]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    """
    # repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev) 
    # concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a,s_prev]) 
    # propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e.
    e = densor1(concat) 
    # propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e) 
    # compute the attention weights "alphas"
    alphas = activator(energies) 
    # compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas,a]) 
    
    return context

In [8]:
def one_step_attention_test(target):

    m = 10
    Tx = 30
    n_a = 32
    n_s = 64
    a = np.random.uniform(1, 0, (m, Tx, 2 * n_a)).astype(np.float32)
    s_prev =np.random.uniform(1, 0, (m, n_s)).astype(np.float32) * 1
    context = target(a, s_prev)

    assert isinstance(context, tf.Tensor), "Unexpected type. It should be a Tensor"
    assert tuple(context.shape) == (m, 1, n_s), "Unexpected output shape"
    assert np.all(context.numpy() > 0), "All output values must be > 0 in this example"
    assert np.all(context.numpy() < 1), "All output values must be < 1 in this example"

    print("\033[92mAll tests passed!")
    
one_step_attention_test(one_step_attention)

All tests passed!


In [9]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [10]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
        
    # Step 1: Define the pre-attention Bi-LSTM.
    a = Bidirectional(LSTM(n_a, return_sequences=True),input_shape=(m, Tx, n_a*2))(X) 
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state]
        s, _, c = post_activation_LSTM_cell(context,initial_state = [s, c] ) 
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    return model

In [11]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

C:\Users\kshitij.singanjude\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 37)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ s0 (InputLayer)               │ (None, 64)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 30, 64)            │          17,920 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ repeat_vector (RepeatVector)  │ (None, 30, 64)            │               0 │ s0[0][0], lstm[0][0],      │
│                               │                           │                 │ lstm[1][0], lstm[2][0],    │
│                               │                           │                 │ lstm[3][0], lstm[4][0],    │
│                               │                           │                 │ lstm[5][0], lstm[6][0],    │
│                               │                           │                 │ lstm[7][0], lstm[8][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 30, 128)           │               0 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[0][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[1][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[2][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[3][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[4][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[5][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[6][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[7][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[8][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[9][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 30, 10)            │           1,29

 Total params: 52,960 (206.88 KB)

 Trainable params: 52,960 (206.88 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999,decay=0.01) 
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'] * Ty)

C:\Users\kshitij.singanjude\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [14]:
assert opt.learning_rate == 0.005, "Set the lr parameter to 0.005"
assert opt.beta_1 == 0.9, "Set the beta_1 parameter to 0.9"
assert opt.beta_2 == 0.999, "Set the beta_2 parameter to 0.999"
assert model.loss == "categorical_crossentropy", "Wrong loss. Use 'categorical_crossentropy'"
assert model.optimizer == opt, "Use the optimizer that you have instantiated"

print("\033[92mAll tests passed!")

All tests passed!


In [15]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [ ]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

 53/100 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - dense_2_accuracy: 0.0361 - dense_2_accuracy_1: 0.3442 - dense_2_accuracy_2: 0.1467 - dense_2_accuracy_3: 0.0516 - dense_2_accuracy_4: 0.7497 - dense_2_accuracy_5: 0.0861 - dense_2_accuracy_6: 0.0255 - dense_2_accuracy_7: 0.7635 - dense_2_accuracy_8: 0.0891 - dense_2_accuracy_9: 0.0450 - dense_2_loss: 2.7338 - loss: 21.4468

In [26]:
model.load_weights('models/model.h5')

In [27]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
source: 3 May 1979
output: 1979-05-33 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

C:\Users\kshitij.singanjude\AppData\Local\Temp\ipykernel_23500\2649609632.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output = [inv_machine_vocab[int(i)] for i in prediction]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
source: 5 April 09
output: 2009-04-05 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
source: 21th of August 2016
output: 2016-08-20 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
source: Tue 10 Jul 2007
output: 2007-07-10 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
source: Saturday May 9 2018
output: 2018-05-09 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
source: March 3 2001
output: 2001-03-03 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
source: March 3rd 2001
output: 2001-03-03 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
source: 1 March 2001
output: 2001-03-01 

